In [1]:
!pip install transformers

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
from tqdm import tqdm
import csv
import sys
import os

In [4]:
torch.cuda.is_available()

True

In [6]:
tokenizer=BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Similarity')
model=BertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Similarity')

In [12]:
texta='今天很开心'
textb='今天心情好'

output=model(torch.tensor([tokenizer.encode(texta,textb)]))
torch.nn.functional.softmax(output.logits,dim=-1)
# 左：不相似概率
# 右：相似概率

tensor([[0.1158, 0.8842]], grad_fn=<SoftmaxBackward0>)

In [8]:
corpus_path = ["data/bq_corpus/test.tsv", "data/lcqmc/test.tsv", "data/paws-x-zh/test.tsv"]
for path in corpus_path[:1]:
    cls_result = []
    test_pairs = open(path, encoding='utf-8').readlines() 
    
    for pair in tqdm(test_pairs):
        pair = pair.split('\t')
        
        output=model(torch.tensor([tokenizer.encode(pair[0],pair[1])]))
        if torch.nn.functional.softmax(output.logits,dim=-1)[0][1] > 0.5:
            cls_result.append(1)
        else:
            cls_result.append(0)
              
    with open('result/' + path.split('/')[1] + '.tsv', 'w') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['index', 'prediction'])
        for i in range(len(cls_result)):
            tsv_writer.writerow([str(i), str(cls_result[i])])

  0%|          | 41/10000 [00:13<54:50,  3.03it/s]  


KeyboardInterrupt: 